In [9]:
import watertap.examples.flowsheets.electrodialysis.electrodialysis_1stack_concentrate_recirc as ef
import importlib
from idaes.core.solvers import get_solver
import pandas as pd
import numpy as np
from pyomo.environ import value, check_optimal_termination
import idaes.logger as idaeslog
from idaes.core.util.exceptions import  InitializationError
from idaes.core.util.model_statistics import report_statistics, degrees_of_freedom, fixed_variables_generator
importlib.reload(ef)


<module 'watertap.examples.flowsheets.electrodialysis.electrodialysis_1stack_concentrate_recirc' from '/Users/xb/Documents/Projects/watertap/watertap/examples/flowsheets/electrodialysis/electrodialysis_1stack_concentrate_recirc.py'>

In [7]:
# To visualize the product salinity as a funciton of applied potential at 
# different feed salinity and water recoveries. 
_log = idaeslog.getLogger(__name__)
try:
    del m
except:
    pass
m = ef.build()
fd_sal=[10] # in g/L
c_mass_fd=pd.DataFrame(fd_sal,columns=["c0_mass"])
#print(c_mass_fd)
initarg_list=ef.make_initarg_list(fd_sal)
#print(initarg_list)
wr_list = [0.5, 0.7, 0.9]
wr=pd.DataFrame(wr_list,columns=["WR"])
volt_list= list(np.arange(2,62,2))
#volt_list=[1,10, 60]
#print(volt_list)
volt=pd.DataFrame(volt_list,columns=["Volt"])
c_wr_v=c_mass_fd.merge(wr,how='cross').merge(volt,how='cross')
#print(c_wr_v)
solver = get_solver()
#deci_var = pd.DataFrame(
 #       columns=["WR", "Voltage"],
  #      index=c_mass_fd["c0_mass"],
   # )
#outdt = pd.DataFrame(
 #       columns=["Prod_Sal", "Brine_Sal"],
  #      index=c_mass_fd["c0_mass"],
   # )
#print(c_wr_v[(c_wr_v["c0_mass"]==1) & (c_wr_v["WR"]==0.7)])
try:
    del outdt
except:
    pass
outdt=c_wr_v.copy()
#print(outdt.loc[2,:])
#for i,r in c_wr_v[(c_wr_v["c0_mass"]==1) & (c_wr_v["WR"]==0.7)].iterrows():
 #   outdt.loc[i,"c_prod_mass"]=r["c0_mass"]
    #print(outdt.loc[i,:])
    #print(f'i is {i}')
    #print(f"r is {r}")
#print(outdt)


#print(c_wr_v.loc(c_wr_v["c0_mass"]==1 and c_wr_v["WR"]==0.7))[(c_wr_v["c0_mass"]==5) & (c_wr_v["WR"]==0.7)]
for i,r in c_wr_v.iterrows():
    print(f"===Excecuting c0_mass= {r['c0_mass']}, WR={r['WR']},V={r['Volt']}===")
    m.fs.feed.find_component("flow_mol_phase_comp").unfix()
    #print("======DOF_intial======")
    #report_statistics(m)
    initarg=ef.make_initarg_list([r["c0_mass"]])
    #print(f"Initial State Var: \n{initarg}")
    deci_var_dict={"voltage_applied[0]":r["Volt"],"recovery_vol_H2O": r["WR"]}
    
    #print(dict(r))
    #print(initarg)
    #print(deci_var_dict)
    #dv=v.to_dict('r')
    #try: 
    try:
        deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
    except:
        _log.warning("Initialization FAILURE at {}; dof is NOT 0! dof={} ".format(r, degrees_of_freedom(m)))
        del m
        m=ef.build()
        #for fc in fixed_variables_generator(m):
            #print(fc)
            #fc.unfix()
        deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
        #pass
    try: 
        solres=solver.solve(m, tee=False)
        if solres.solver.termination_condition=="maxIterations":
            solres=solver.solve(m,tee=False)
        if check_optimal_termination(solres):
            outdt.loc[i,"c_prod_mass"]= value(m.fs.prod_salinity)
            outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
        else:
    #except:
            _log.warning("Solving FAILING at {} with solver status of {}"
                         " and termination condition of {} ".format(
                             r,solres.solver.status, solres.solver.termination_condition))
            outdt.loc[i,"c_prod_mass"]= float("nan")
            outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
    except ValueError as err:
        _log.warning(err)
        outdt.loc[i,"c_prod_mass"]= float("nan")
        outdt.loc[i,"Sol_Cond"]="error"
    for comp in deci_comp:
        comp.unfix()
    #print("======DOF_after======")
    #report_statistics(m)
 
print(outdt)
outdt.to_csv("~/Documents/Projects/wt_untracked/output290323/opt_prod_sal_volt_c0_10_wr_2_62.csv")


 

"""

init_arg = {
    ("flow_vol_phase", ("Liq")): 5.2e-4,
    ("conc_mol_phase_comp", ("Liq", "Na_+")): 34.188,
    ("conc_mol_phase_comp", ("Liq", "Cl_-")): 34.188,
}  # Corresponding to C_feed = 2g/L
m.fs.feed.properties.calculate_state(
    init_arg,
    hold_state=True,
)
m.fs.EDstack.voltage_applied[0].fix(10)
m.fs.recovery_vol_H2O.fix(0.7)
edfs.condition_base(m)
check_dof(m)
"""

2023-04-17 15:08:18 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-17 15:08:18 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
===Excecuting c0_mass= 10.0, WR=0.5,V=2.0===
2023-04-17 15:08:18 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable flow_vol_phase was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an ini

2023-04-17 15:10:24 [INFO] idaes.init.fs.pump1: Initialization Complete: optimal - Optimal Solution Found
2023-04-17 15:10:24 [WARNING] idaes.__main__: Initialization FAILURE at c0_mass    10.0
WR          0.5
Volt        6.0
Name: 2, dtype: float64; dof is NOT 0! dof=-5 
2023-04-17 15:10:24 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-17 15:10:24 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-17 15:10:24 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable flow_vol_phase was provided as an argument in var_args, but it does not have a scaling factor. This 

KeyboardInterrupt: 

In [6]:
df1=pd.read_csv('dt/opt_prod_sal_volt_c0_wr.csv', index_col=0)
df2=pd.read_csv('dt/opt_prod_sal_volt_c0_wr_30_80.csv', index_col=0)
df3=pd.read_csv('dt/opt_prod_sal_volt_c0_wr_60_100.csv', index_col=0)
df_conc=pd.concat([df1, df2, df3], ignore_index=True,sort=False).drop_duplicates(["c0_mass", "WR", "Volt"], keep="last", ignore_index=True)
#print(df1)
#print(df2)
#print(df3)
#print(135+225+180)
df_conc.sort_values(by=["c0_mass","WR", "Volt" ], inplace= True,ascending=True, ignore_index=True)#, "WR", "Volt"
#df_conc.sort_values?
print(df_conc)
#df_conc.drop("Unnamed: 0", axis=1)
#df_conc.drop_duplicates(["c0_mass", "WR", "Volt"], keep="last")
df_conc.to_csv("dt/opt_prod_sal_volt_c0_wr_combined.csv")
#print(df_conc)


     c0_mass   WR  Volt  c_prod_mass    Sol_Cond
0          1  0.5     2     0.894800     optimal
1          1  0.5     4     0.794034     optimal
2          1  0.5     6     0.699591     optimal
3          1  0.5     8     0.612840     optimal
4          1  0.5    10     0.534571     optimal
..       ...  ...   ...          ...         ...
445       15  0.9    92          NaN  infeasible
446       15  0.9    94          NaN  infeasible
447       15  0.9    96          NaN  infeasible
448       15  0.9    98          NaN  infeasible
449       15  0.9   100          NaN  infeasible

[450 rows x 5 columns]


In [74]:
_log = idaeslog.getLogger(__name__)
solver=get_solver()
df=pd.read_csv('dt/opt_prod_sal_volt_c0_wr_combined.csv', index_col=0)
try:
    del m
except:
    pass
m = ef.build()
try:
    del outdt1
except: 
    pass
outdt1=df.copy()
print(df.loc[6,"c_prod_mass"])
#print(pd.isna(df.loc[5, "c_prod_mass"])) #and (df.loc[6-1, "c_prod_mass"]!=float("nan") or df.loc[6+1, "c_prod_mass"]!=float("nan")))
for i,r in df.iterrows():
    
    if (pd.isna(df.loc[i, "c_prod_mass"]) 
        and df.loc[i,"Volt"]!=100
        and (
            not pd.isna(df.loc[i-1, "c_prod_mass"])
            or 
            not pd.isna(df.loc[i+1, "c_prod_mass"]))):
        
        print(f"===Excecuting c0_mass= {r['c0_mass']}, WR={r['WR']},V={r['Volt']}===")
        m.fs.feed.find_component("flow_mol_phase_comp").unfix()
        #print("======DOF_intial======")
        #report_statistics(m)
        initarg=ef.make_initarg_list([r["c0_mass"]])
        #print(f"Initial State Var: \n{initarg}")
        deci_var_dict={"voltage_applied[0]":r["Volt"],"recovery_vol_H2O": r["WR"]}

        #print(dict(r))
        #print(initarg)
        #print(deci_var_dict)
        #dv=v.to_dict('r')
        #try: 
        try:
            deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
        except:
            _log.warning("Initialization FAILURE at {}; dof is NOT 0! dof={} ".format(r, degrees_of_freedom(m)))
            del m
            m=ef.build()
            #for fc in fixed_variables_generator(m):
                #print(fc)
                #fc.unfix()
            deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
            #pass
        try: 
            solres=solver.solve(m, tee=False)
            if solres.solver.termination_condition=="maxIterations":
                solres=solver.solve(m,tee=False)
            if check_optimal_termination(solres):
                outdt1.loc[i,"c_prod_mass"]= value(m.fs.prod_salinity)
                outdt1.loc[i,"Sol_Cond"]=solres.solver.termination_condition
            else:
        #except:
                _log.warning("Solving FAILING at {} with solver status of {}"
                             " and termination condition of {} ".format(
                                 r,solres.solver.status, solres.solver.termination_condition))
                outdt1.loc[i,"c_prod_mass"]= float("nan")
                outdt1.loc[i,"Sol_Cond"]=solres.solver.termination_condition
        except ValueError as err:
            _log.warning(err)
            outdt1.loc[i,"c_prod_mass"]= float("nan")
            outdt1.loc[i,"Sol_Cond"]="error"
        for comp in deci_comp:
            comp.unfix()
        
outdt1.to_csv("dt/opt_prod_sal_volt_c0_wr_rerun_missing_dt2.csv")


NameError: name 'idaeslog' is not defined

In [1]:
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd
import kaleido


In [6]:
df0=pd.read_csv("dt/opt_prod_sal_volt_c0_wr_rerun_missing_dt2.csv", index_col=0)
#print(df_p)
#ex.line?
#print(df0[(df0["c0_mass"]==1) & (df0["WR"]==0.5)]["c_prod_mass"])
#fig0=go.Figure()
layout = dict(
xaxis = dict(title = "Voltage (V)", range = [0, 60], mirror=True),
yaxis= dict (title = "Product salinity (g/L)", range = [0, 1], mirror = True),
legend=dict(
        x=.8,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        )),
height=400,
template="simple_white",

#showline=True
)

# C0_mass = 1g/L
c1w50=go.Scatter(
    x=df0[df0["c0_mass"]==1]["Volt"],
    y=df0[(df0["c0_mass"]==1)&(df0["WR"]==0.5)]["c_prod_mass"],
    mode = 'lines',
    #type = "scatter",
    name = 'WR=0.5',
    line = dict(shape = 'spline', color = 'black', width =2,),
    connectgaps=True
),
c1w70=go.Scatter(
    x=df0[df0["c0_mass"]==1]["Volt"],
    y=df0[(df0["c0_mass"]==1)&(df0["WR"]==0.7)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.7',
    line = dict(shape = 'spline', color = 'red', width =2,),
    connectgaps=True
),
c1w90=go.Scatter(
    x=df0[df0["c0_mass"]==1]["Volt"],
    y=df0[(df0["c0_mass"]==1)&(df0["WR"]==0.9)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.9',
    line = dict(shape = 'spline', color = 'blue', width =2,),
    connectgaps=True
),

data0 = [c1w50[0], c1w70[0],c1w90[0]]
fig0=go.Figure(data=data0,layout=layout)
fig0.update_layout(title= dict(text= "Feed = 1g/L", x=0.5, y = 0.7),
                  )



# C0_mass = 5g/L
c5w50=go.Scatter(
    x=df0[df0["c0_mass"]==5]["Volt"],
    y=df0[(df0["c0_mass"]==5)&(df0["WR"]==0.5)]["c_prod_mass"],
    mode = 'lines',
    #type = "scatter",
    name = 'WR=0.5',
    line = dict(shape = 'spline', color = 'black', width =2,),
    connectgaps=True
),
c5w70=go.Scatter(
    x=df0[df0["c0_mass"]==5]["Volt"],
    y=df0[(df0["c0_mass"]==5)&(df0["WR"]==0.7)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.7',
    line = dict(shape = 'spline', color = 'red', width =2,),
    connectgaps=True
),
c5w90=go.Scatter(
    x=df0[df0["c0_mass"]==5]["Volt"],
    y=df0[(df0["c0_mass"]==5)&(df0["WR"]==0.9)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.9',
    line = dict(shape = 'spline', color = 'blue', width =2,),
    connectgaps=True
),


data1 = [c5w50[0], c5w70[0],c5w90[0]]
fig1=go.Figure(data=data1,layout=layout)
fig1.update_layout(title= dict(text= "Feed = 5g/L", x=0.5, y = 0.7),
                  xaxis = dict(title = "Voltage (V)", range = [0, 60], mirror=True),
                yaxis= dict (title = "Product salinity (g/L)", range = [0, 5], mirror = True),)

# C0_mass = 15g/L
c15w50=go.Scatter(
    x=df0[df0["c0_mass"]==15]["Volt"],
    y=df0[(df0["c0_mass"]==15)&(df0["WR"]==0.5)]["c_prod_mass"],
    mode = 'lines',
    #type = "scatter",
    name = 'WR=0.5',
    line = dict(shape = 'spline', color = 'black', width =2,),
    connectgaps=True
),
c15w70=go.Scatter(
    x=df0[df0["c0_mass"]==15]["Volt"],
    y=df0[(df0["c0_mass"]==15)&(df0["WR"]==0.7)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.7',
    line = dict(shape = 'spline', color = 'red', width =2,),
    connectgaps=True
),
c15w90=go.Scatter(
    x=df0[df0["c0_mass"]==15]["Volt"],
    y=df0[(df0["c0_mass"]==15)&(df0["WR"]==0.9)]["c_prod_mass"],
    mode = 'lines',
    name = 'WR=0.9',
    line = dict(shape = 'spline', color = 'blue', width =2,),
    connectgaps=True
),


data2 = [c15w50[0], c15w70[0],c15w90[0]]
fig2=go.Figure(data=data2,layout=layout)
fig2.update_layout(title= dict(text= "Feed = 15g/L", x=0.5, y = 0.7),
                  xaxis = dict(title = "Voltage (V)", range = [0, 60], mirror=True),
                yaxis= dict (title = "Product salinity (g/L)", range = [0, 15], mirror = True),)
"""
fig0 = go.line(df_p[df_p["c0_mass"]==1], 
               x="Volt", y="c_prod_mass", 
               labels={"Volt":"Voltage (V)", "c_prod_mass":"Product water salinity (g/L)"},
               range_x=[0, 60],
               range_y=[0, 1],
               
               color="WR", line_shape="spline", template="simple_white" )#
fig0.update_traces(connectgaps=True)
"""
fig0.show()
fig1.show()
fig2.show()
fig0.write_image("graphics/cprod_volt_at_cf_1.png")
fig1.write_image("graphics/cprod_volt_at_cf_5.png")
fig2.write_image("graphics/cprod_volt_at_cf_15.png")

In [10]:
# To visualize the product salinity as a funciton of applied potential at 
# different feed salinity and water recoveries. 
_log = idaeslog.getLogger(__name__)
try:
    del m
except:
    pass
#m = ef.build()
fd_sal=[1] # in g/L
c_mass_fd=pd.DataFrame(fd_sal,columns=["c0_mass"])
initarg_list=ef.make_initarg_list(fd_sal)
wr_list = [0.5]
wr=pd.DataFrame(wr_list,columns=["WR"])
volt_list= list(np.arange(2,62,2))
volt=pd.DataFrame(volt_list,columns=["Volt"])
c_wr_v=c_mass_fd.merge(wr,how='cross').merge(volt,how='cross')
solver = get_solver()
try:
    del outdt
except:
    pass
outdt=c_wr_v.copy()
for i,r in c_wr_v.iterrows():
    print(f"===Excecuting c0_mass= {r['c0_mass']}, WR={r['WR']},V={r['Volt']}===")
    if (i>1 and
        outdt.loc[i-1, "c0_mass"]==outdt.loc[i, "c0_mass"]
        and
        outdt.loc[i-1, "WR"]==outdt.loc[i, "WR"]
        and
        pd.isna(outdt.loc[i-1, "c_prod_mass"])
        and pd.isna(outdt.loc[i-2, "c_prod_mass"])):
        
        outdt.loc[i,"c_prod_mass"]= float("nan")
        print("Consective Fails, pass solving!!")
        continue                                                 
                                                      
    m=ef.build()
    print("======DOF_intial======")
    report_statistics(m)
    initarg=ef.make_initarg_list([r["c0_mass"]])
    #print(f"Initial State Var: \n{initarg}")
    deci_var_dict={"voltage_applied[0]":r["Volt"],"recovery_vol_H2O": r["WR"]}
    
    deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
    try: 
        solres=ef.solve(m, tee=False,check_termination=True)
    except:
        if solres.solver.termination_condition=="maxIterations":
            solres=solver.solve(m,tee=False)
            if check_optimal_termination(solres):
                outdt.loc[i,"c_prod_mass"]= value(m.fs.prod_salinity)
                outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
        else:
    #except:
            _log.warning("Solving FAILING at {} with solver status of {}"
                         " and termination condition of {} ".format(
                             r,solres.solver.status, solres.solver.termination_condition))
            outdt.loc[i,"c_prod_mass"]= float("nan")
            outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
            
    outdt.loc[i,"c_prod_mass"]= value(m.fs.prod_salinity)
    outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
    outdt.to_csv("~/Documents/Projects/wt_untracked/output290323/opt_prod_sal_volt_c0_wr_2_102_d_qc2.csv")
    
    #for comp in deci_comp:
        #comp.unfix()
    #print("======DOF_after======")
    #report_statistics(m)
 
print(outdt)
outdt.to_csv("~/Documents/Projects/wt_untracked/output290323/opt_prod_sal_volt_c0_wr_2_102_d_qc2.csv")


 

"""

init_arg = {
    ("flow_vol_phase", ("Liq")): 5.2e-4,
    ("conc_mol_phase_comp", ("Liq", "Na_+")): 34.188,
    ("conc_mol_phase_comp", ("Liq", "Cl_-")): 34.188,
}  # Corresponding to C_feed = 2g/L
m.fs.feed.properties.calculate_state(
    init_arg,
    hold_state=True,
)
m.fs.EDstack.voltage_applied[0].fix(10)
m.fs.recovery_vol_H2O.fix(0.7)
edfs.condition_base(m)
check_dof(m)
"""

===Excecuting c0_mass= 1.0, WR=0.5,V=2.0===
2023-04-26 17:17:25 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:17:25 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:17:26 [WARNING] idaes.watertap.prope

===Excecuting c0_mass= 1.0, WR=0.5,V=4.0===
2023-04-26 17:17:40 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:17:40 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:17:40 [WARNING] idaes.watertap.prope

===Excecuting c0_mass= 1.0, WR=0.5,V=6.0===
2023-04-26 17:18:28 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:18:28 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:18:28 [WARNING] idaes.watertap.prope

===Excecuting c0_mass= 1.0, WR=0.5,V=8.0===
2023-04-26 17:18:46 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:18:46 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:18:46 [WARNING] idaes.watertap.prope

===Excecuting c0_mass= 1.0, WR=0.5,V=10.0===
2023-04-26 17:18:51 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:18:51 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:18:51 [WARNING] idaes.watertap.prop

...

===Excecuting c0_mass= 1.0, WR=0.5,V=54.0===
2023-04-26 17:23:19 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:23:19 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:23:19 [WARNING] idaes.watertap.prop

===Excecuting c0_mass= 1.0, WR=0.5,V=56.0===
2023-04-26 17:23:29 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-04-26 17:23:29 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
======DOF_intial======

Model Statistics   

Degrees of Freedom: 38 

Total No. Variables: 2374 
    No. Fixed Variables: 24
    No. Unused Variables: 16 (Fixed):5)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 2301 
    No. Equality Constraints: 2301 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 0 (Deactivated: 0)

No. Blocks: 94 (Deactivated: 0) 
No. Expressions: 10 

2023-04-26 17:23:29 [WARNING] idaes.watertap.prop

2023-04-26 17:24:03 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable conc_mol_phase_comp was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an initial value) before using the calculate_scaling_factor method.
2023-04-26 17:24:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.EDstack.diluate.area
2023-04-26 17:24:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.EDstack.concentrate.area
2023-04-26 17:24:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.EDstack.membrane_thickness
2023-04-26 17:24:03 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.EDstack.membrane_areal_resi

2023-04-26 17:24:16 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable conc_mol_phase_comp was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an initial value) before using the calculate_scaling_factor method.
2023-04-26 17:24:16 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable conc_mol_phase_comp was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to 

'\n\ninit_arg = {\n    ("flow_vol_phase", ("Liq")): 5.2e-4,\n    ("conc_mol_phase_comp", ("Liq", "Na_+")): 34.188,\n    ("conc_mol_phase_comp", ("Liq", "Cl_-")): 34.188,\n}  # Corresponding to C_feed = 2g/L\nm.fs.feed.properties.calculate_state(\n    init_arg,\n    hold_state=True,\n)\nm.fs.EDstack.voltage_applied[0].fix(10)\nm.fs.recovery_vol_H2O.fix(0.7)\nedfs.condition_base(m)\ncheck_dof(m)\n'

In [171]:
volt_list= list(np.arange(2, 62, 2))
print(volt_list)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60]
